In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
dbutils.fs.ls("abfss://landing@salesmarketingstorage1.dfs.core.windows.net")

In [0]:
df = spark.read \
.options(header=True) \
.csv("abfss://landing@salesmarketingstorage1.dfs.core.windows.net/sellers")

df.display(10)

In [0]:
df_sellers = (
    spark.readStream 
    .format("cloudFiles").option("cloudFiles.format", "csv")
    .option("header", "true")
    .option("quote", "\"")
    .option("mode", "PERMISSIVE")
    .option("cloudFiles.schemaLocation", "abfss://metadata@salesmarketingstorage1.dfs.core.windows.net/bronze/sellers")
    .load("abfss://landing@salesmarketingstorage1.dfs.core.windows.net/sellers/")
    .writeStream
    .format("delta")
    .option("checkpointLocation", "abfss://metadata@salesmarketingstorage1.dfs.core.windows.net/checkpoints/bronze/sellers") 
    .table("bronze.sellers")
)

In [0]:
%sql
describe sales_marketing_ws.bronze.sellers;

In [0]:
%sql
SELECT * FROM cloud_files_state('abfss://metadata@salesmarketingstorage1.dfs.core.windows.net/checkpoints/bronze/sellers');

In [0]:
%sql
select * from sales_marketing_ws.bronze.sellers limit 10;